# Trabalho Final - Aprendizado não supervisionado
## Aluno: Diego Leonardo Urban

## Importando as dependências

In [1]:
# Python
import sys
import random
from os import listdir
from os.path import isfile, join

# Numpy
import numpy as np

# OpenCV
import cv2
from google.colab.patches import cv2_imshow

## Definição das Classes

In [2]:
# https://github.com/lobokoch/unsupervised-learning2/blob/main/pca-eigenfaces/src/br/furb/Person.java
class Person():
  """Classe responsável por representar os dados de uma pessoa"""
  
  def __init__(self, id, label, data):
    self.id = id
    self.label = label
    self.data = data

In [3]:
# https://github.com/lobokoch/unsupervised-learning2/blob/main/pca-eigenfaces/src/br/furb/PCA.java
class PCA():
  """Classe reponsável por realizar as operações de PCA"""

  #@classmethod
  def image_data(self, filename):
      img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
      
      dsize = (80, 80)
      output = cv2.resize(img, dsize)

      dst = output.T.reshape((1, output.shape[1] * output.shape[0]))
      return np.float64(dst)

  #@classmethod
  def to_person(self, filename):
    data_part = filename[filename.rfind('/')+1 :filename.rfind('.jpg')]
    data = data_part.split('_')

    return Person(int(data[0]), int(data[1]), self.image_data(filename))

  #@classmethod
  def load_dataset(self, path, train, test, p):
    onlyfiles = [f for f in listdir('./') if isfile(join('./', f)) if f.endswith('.jpg')]
    people = list(map(self.to_person, onlyfiles))

    people.sort(key=lambda x: x.id, reverse=True)

    num_samples_per_person = 10
    samples = []
    for person in people:
      samples.append(person)
      if len(samples) == num_samples_per_person:
        while len(samples) > p:
          index = random.randint(0, len(samples)-1)          
          test.append(samples.pop(index))

        if p == num_samples_per_person:
          test.extend(samples)

        train.extend(samples)
        samples = []

  def main(self):
    path = './'
    train = []
    test = []
    p = 7

    self.load_dataset(path, train, test, p)

    start = 2
    end = 30

    MAX_REC = 3500
    MAX_DIS = 1700

    for k in range(start, end):
      recognizer = cv2.face.EigenFaceRecognizer_create(k)
      src = []
      labels = []
      for person_train in train:
        src.append(person_train.data)
        labels.append(person_train.label)

      recognizer.train(src, np.asarray(labels))

      min_distance = sys.float_info.max
      max_distance = sys.float_info.min

      true_negative_count = 0
      true_positive_count = 0

      corrects = 0

      for person_test in test:
        label, confidence = recognizer.predict(person_test.data)
        
        label_ok = label == person_test.label
        if label_ok:
          corrects = corrects + 1

        if confidence > max_distance:
          if not label_ok:
            true_negative_count = true_negative_count + 1
        elif confidence > 1500:
          if not label_ok:
            true_negative_count = true_negative_count + 1
        elif label_ok:
          true_positive_count = true_positive_count + 1

        if label_ok and person_test.id <= 400:
          if confidence < min_distance:
            min_distance = confidence

          if confidence > max_distance:
            max_distance = confidence

      print(f"K: {k}")
      print(f"Corrects: {corrects}")
      trues = true_positive_count + true_negative_count
      accuracy = trues / len(test) * 100
      print(f"Accuracy: {accuracy}")
      print(f"Min. Distance: {min_distance}")
      print(f"Max. Distance: {max_distance}")
      print("-------------------")

## Execução

In [4]:
print('Welcome to the PCA Tool!')

pca = PCA()
pca.main()

Welcome to the PCA Tool!
K: 2
Corrects: 47
Accuracy: 37.5
Min. Distance: 0.8726075525811382
Max. Distance: 304.88531619552333
-------------------
K: 3
Corrects: 95
Accuracy: 77.5
Min. Distance: 43.76448471838347
Max. Distance: 728.4547711009322
-------------------
K: 4
Corrects: 105
Accuracy: 84.16666666666667
Min. Distance: 87.1434040519468
Max. Distance: 902.031738520689
-------------------
K: 5
Corrects: 104
Accuracy: 84.16666666666667
Min. Distance: 89.43713392970966
Max. Distance: 1015.3871844325666
-------------------
K: 6
Corrects: 110
Accuracy: 86.66666666666667
Min. Distance: 97.85215960927641
Max. Distance: 1110.8729865587986
-------------------
K: 7
Corrects: 112
Accuracy: 90.0
Min. Distance: 99.91475940411694
Max. Distance: 1150.7445747031
-------------------
K: 8
Corrects: 114
Accuracy: 90.0
Min. Distance: 99.97519817755185
Max. Distance: 1662.9763055286007
-------------------
K: 9
Corrects: 115
Accuracy: 90.83333333333333
Min. Distance: 100.79013433020499
Max. Distance: 1